In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import csc665.tree

# Use Instructor's code for basic functions to avoid various inconsistencies
# For Students: you can change this to use YOUR code if you fix any errors.
from csc665 import features, metrics

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import unittest as ut
import sklearn.tree

class Object(object): pass
class TC(ut.TestCase): pass
tc = TC()

In [4]:
csv_df = pd.read_csv("/Users/patrick/Documents/csc665/Melbourne_housing_FULL.csv")

In [5]:
_mel = features.Object()
_mel.X, _mel.y = features.preprocess_ver_1(csv_df, "Price")

In [6]:
print(_mel.X.shape, _mel.y.shape)

assert _mel.X.shape[0] == 8887, "X.shape"
assert _mel.y.shape[0] == 8887, "y.shape"

(8887, 20) (8887,)


In [7]:
_mel.X_train, _mel.X_test, _mel.y_train, _mel.y_test = features.train_test_split(
    _mel.X, _mel.y, test_size=0.2, shuffle=True, random_state=None)

In [8]:
print(_mel.X.shape, _mel.X_train.shape, _mel.X_test.shape, _mel.y_train.shape, _mel.y_test.shape)

tc.assertEqual(_mel.X_test.shape[0], _mel.y_test.shape[0])
tc.assertEqual(_mel.X_train.shape[0], _mel.y_train.shape[0])

(8887, 20) (7109, 20) (1778, 20) (7109,) (1778,)


In [9]:
if "DataFrame" in str(type(_mel.y_train)):
    _mel.y_train = _mel.y_train["Price"].values

if "DataFrame" in str(type(_mel.y_test)):
    _mel.y_test = _mel.y_test["Price"].values

In [10]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100 , random_state=17)
%time _ = rf.fit(_mel.X_train, _mel.y_train)

CPU times: user 4.72 s, sys: 39.3 ms, total: 4.76 s
Wall time: 4.77 s


In [11]:
_st = Object()
_act = Object()

_st.mse = metrics.mse(rf.predict(_mel.X_test), _mel.y_test)
_act.mse = mean_squared_error(_mel.y_test, rf.predict(_mel.X_test))

print(_st.mse, _act.mse)

tc.assertAlmostEqual(_st.mse, _act.mse, delta=1)

64859701329.99842 64859701329.99842


In [12]:
_st.r2_test = metrics.rsq(rf.predict(_mel.X_test), _mel.y_test)
_act.r2_test = r2_score(_mel.y_test, rf.predict(_mel.X_test))

print(_st.r2_test, _act.r2_test)
tc.assertAlmostEqual(_st.r2_test, _act.r2_test, places=3)
tc.assertGreater(_st.r2_test, 0.6)

0.83954319791047 0.8395431979104702


In [13]:

_st.r2_train = metrics.rsq(rf.predict(_mel.X_train), _mel.y_train)
_act.r2_train = r2_score(_mel.y_train, rf.predict(_mel.X_train))

print(_st.r2_train, _act.r2_train)
tc.assertAlmostEqual(_st.r2_train, _act.r2_train, places=3)
tc.assertGreater(_st.r2_train, 0.6)

0.9731946349837682 0.9731946349837682


In [14]:
def compare_trees(sktree, our_tree, X_train, X_test, y_train, y_test):
    sktree.fit(X_train, y_train)
    our_tree.fit(X_train, y_train)
#     print(our_tree)
    
    expected = metrics.rsq(sktree.predict(X_train), y_train)
    actual = metrics.rsq(our_tree.predict(X_train), y_train)
    print("Train, sklearn, ours:", expected, actual)
    tc.assertAlmostEqual(expected, actual, places=1)
    
    expected = metrics.rsq(sktree.predict(X_test), y_test)
    actual = metrics.rsq(our_tree.predict(X_test), y_test)
    print("Test, sklearn, ours:", expected, actual)
    tc.assertLess(abs(expected - actual), 0.19)

In [15]:
income_df = pd.read_csv("http://www-bcf.usc.edu/~gareth/ISL/Income2.csv")
income_df = income_df.drop("Unnamed: 0", axis=1)
_income = features.Object()
_income.X = income_df.drop("Income", axis=1)
_income.y = income_df["Income"].values

In [16]:
RAND_STATE = 17

_income.X_train, _income.X_test, _income.y_train, _income.y_test = \
    features.train_test_split(
        _income.X, 
        _income.y, 
        test_size=0.2, 
        shuffle=True, 
        random_state=RAND_STATE)

In [17]:
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=2, min_samples_leaf=1),
    csc665.tree.DecisionTreeRegressor(max_depth=2, min_samples_leaf=1),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

Train, sklearn, ours: 0.9196882067371206 0.9196882067371206
Test, sklearn, ours: 0.7025467606977926 0.7906036019945054


In [18]:
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=100, min_samples_leaf=1),
    csc665.tree.DecisionTreeRegressor(max_depth=100, min_samples_leaf=1),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

Train, sklearn, ours: 1.0 1.0
Test, sklearn, ours: 0.878556977679157 0.9647026064064674


In [19]:
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=5, min_samples_leaf=1),
    csc665.tree.DecisionTreeRegressor(max_depth=5, min_samples_leaf=1),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

Train, sklearn, ours: 0.9992698428541422 0.9992698428541422
Test, sklearn, ours: 0.956126168903839 0.9627112963594541


In [20]:
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=20, min_samples_leaf=5),
    csc665.tree.DecisionTreeRegressor(max_depth=20, min_samples_leaf=5),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

Train, sklearn, ours: 0.900021992414658 0.900021992414658
Test, sklearn, ours: 0.6972233317017924 0.7852801729985053


In [21]:
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=20, min_samples_leaf=7),
    csc665.tree.DecisionTreeRegressor(max_depth=20, min_samples_leaf=7),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

Train, sklearn, ours: 0.8647154870659806 0.8647154870659806
Test, sklearn, ours: 0.7054294009082782 0.793486242204991


In [22]:
_mel_sampled = features.Object()
_mel_sampled.X_train = _mel.X_train[:200]
_mel_sampled.y_train = _mel.y_train[:200]
_mel_sampled.X_test = _mel.X_test[:50]
_mel_sampled.y_test = _mel.y_test[:50]

In [23]:
dt = csc665.tree.DecisionTreeRegressor(max_depth=100, min_samples_leaf=1)

In [24]:
%%prun
dt.fit(_mel_sampled.X_train, _mel_sampled.y_train)

In [25]:
%%time 
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=5, min_samples_leaf=1),
    csc665.tree.DecisionTreeRegressor(max_depth=5, min_samples_leaf=1),
    _mel_sampled.X_train, _mel_sampled.X_test, _mel_sampled.y_train, _mel_sampled.y_test)

Train, sklearn, ours: 0.8987998210692171 0.8987998210692171
Test, sklearn, ours: 0.29174211706822906 0.2965592084589944
CPU times: user 997 ms, sys: 4.46 ms, total: 1 s
Wall time: 1 s


In [26]:
%%time 
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=100, min_samples_leaf=1),
    csc665.tree.DecisionTreeRegressor(max_depth=100, min_samples_leaf=1),
    _mel_sampled.X_train, _mel_sampled.X_test, _mel_sampled.y_train, _mel_sampled.y_test)

Train, sklearn, ours: 1.0 1.0
Test, sklearn, ours: -0.13531037651964017 -0.027470394746961446
CPU times: user 2.44 s, sys: 8.23 ms, total: 2.45 s
Wall time: 2.45 s


In [27]:
%%time 
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=100, min_samples_leaf=7),
    csc665.tree.DecisionTreeRegressor(max_depth=100, min_samples_leaf=7),
    _mel_sampled.X_train, _mel_sampled.X_test, _mel_sampled.y_train, _mel_sampled.y_test)

Train, sklearn, ours: 0.8165200588363422 0.8165200588363422
Test, sklearn, ours: 0.3054985301895722 0.3054985301895722
CPU times: user 1.33 s, sys: 3.86 ms, total: 1.33 s
Wall time: 1.33 s
